# YouTube 音訊轉文字工具

這個 notebook 可以：
- 下載 YouTube 影片的音訊
- 使用 Whisper 模型將音訊轉換成中文文字
- 支援單一影片或整個播放清單
- 輸出包含時間戳記的文字檔

## 修改使用 GPU 運算
執行階段 -> 變更執行類型 -> 選T4 TPU


## 使用方法
1. 依序執行每個 cell
2. 在指定的 cell 中修改 YouTube URL
3. 等待處理完成後下載結果

## 1. 安裝必要套件

In [ ]:
# 安裝所需套件
!pip install yt-dlp
!pip install openai-whisper
!pip install torch torchvision torchaudio
!pip install keyring browser-cookie3


## 2. 匯入套件和初始設定

In [ ]:
import yt_dlp
import whisper
import glob
import os
from urllib.parse import urlparse, parse_qs

# 建立必要資料夾
os.makedirs("./subtitles/", exist_ok=True)
os.makedirs("./download_yt/", exist_ok=True)

print("資料夾建立完成！")

## 3. 載入 Whisper 模型

In [ ]:
# 載入 Whisper 模型 (第一次會需要下載模型檔案)
print("正在載入 Whisper 模型...")
model = whisper.load_model("medium")
print("模型載入完成！")

# 設定路徑
export_dir = "./subtitles/"
download_dir = "./download_yt/"

## 4. 定義核心函數

In [ ]:
# 使用這個插件下載 cookies.txt 並且使用 左邊上傳圖示上傳到這個目錄下
# https://chromewebstore.google.com/detail/get-cookiestxt-locally/cclelndahbckbenkjhflpdbgdldlbecc

In [ ]:
def download_audio_clip(url, output_dir="./download_yt/"):
    ydl_opts = {
        "format": "bestaudio",
        "outtmpl": output_dir + "%(title)s.%(ext)s",
        "noplaylist": True,
        "cookiefile": "cookies.txt",
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
    return info
def get_playlist_urls(playlist_url):
    """取得播放清單中所有影片的 URL"""
    def extract_list_id(url: str) -> str | None:
        parsed = urlparse(url)
        qs = parse_qs(parsed.query)
        return qs.get("list", [None])[0]

    play_list_id = extract_list_id(playlist_url)
    if not play_list_id:
        print("錯誤：無效的播放清單 URL")
        return []
    else:
        playlist_url = f"https://www.youtube.com/playlist?list={play_list_id}"

    ydl_opts = {
        "quiet": True,
        "extract_flat": True,
        "skip_download": True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(playlist_url, download=False)
        return [entry["url"] for entry in info.get("entries", []) if "url" in entry]

def create_txt(fn, verbose: bool = False):
    """將音訊檔案轉換成文字檔"""
    print(f"正在轉換: {fn}")
    result = model.transcribe(fn, language="zh", verbose=verbose)

    # 修正路徑分隔符號問題 (適用於不同作業系統)
    filename = os.path.basename(fn)
    export_fn = os.path.join(export_dir, filename.rsplit('.', 1)[0] + '.txt')

    def format_timestamp(seconds):
        seconds = int(seconds)
        minutes, seconds = divmod(seconds, 60)
        hours, minutes = divmod(minutes, 60)
        return f"{hours:02}:{minutes:02}:{seconds:02}"

    with open(export_fn, "w", encoding="utf-8") as file:
        for segment in result["segments"]:
            start = format_timestamp(segment["start"])
            end = format_timestamp(segment["end"])
            text = segment["text"]
            file.write(f"[{start} --> {end}] {text}\n")

    print(f"轉換完成: {export_fn}")

print("函數定義完成！")

## 5. 方法一：下載指定影片

在下面的 cell 中修改 `url_list` 來指定要下載的影片

In [ ]:
# 🔽 在這裡修改你要下載的影片 URL
url_list = [
    "https://www.youtube.com/watch?v=h5d9rsBkd4Y&list=PLlk-i0VfFy44iTM8hWIUJ0ZLXh0fAFYeK&index=2",
    "https://www.youtube.com/watch?v=ReAe7QMzj8A&list=PLlk-i0VfFy44iTM8hWIUJ0ZLXh0fAFYeK&index=3",
]

# 下載音訊
print("開始下載音訊...")
for i, url in enumerate(url_list, 1):
    try:
        print(f"下載第 {i}/{len(url_list)} 個影片...")
        download_audio_clip(url)
        print(f"✅ 下載完成: {url}")
    except Exception as e:
        print(f"❌ 下載失敗: {url}, 錯誤: {e}")

# 轉換成文字
print("\n開始轉換音訊為文字...")
video_list = glob.glob("./download_yt/*.mp4") + glob.glob("./download_yt/*.webm")
for i, fn in enumerate(video_list, 1):
    print(f"轉換第 {i}/{len(video_list)} 個檔案...")
    create_txt(fn)

print("\n🎉 所有處理完成！")

## 6. 方法二：下載整個播放清單

如果要下載整個播放清單，請執行下面的 cell

In [ ]:
# 🔽 在這裡修改播放清單 URL
playlist_url = "https://www.youtube.com/playlist?list=PLlk-i0VfFy44iTM8hWIUJ0ZLXh0fAFYeK"

# 取得播放清單中的所有影片 URL
print("正在取得播放清單...")
url_list = get_playlist_urls(playlist_url)
print(f"找到 {len(url_list)} 個影片")

if url_list:
    # 下載所有影片的音訊
    print("\n開始下載音訊...")
    for i, url in enumerate(url_list, 1):
        try:
            print(f"下載第 {i}/{len(url_list)} 個影片...")
            download_audio_clip(url)
            print(f"✅ 下載完成")
        except Exception as e:
            print(f"❌ 下載第 {i} 個影片失敗: {e}")

    # 轉換所有音訊為文字
    print("\n開始轉換音訊為文字...")
    video_list = glob.glob("./download_yt/*.mp4") + glob.glob("./download_yt/*.webm")
    for i, fn in enumerate(video_list, 1):
        print(f"轉換第 {i}/{len(video_list)} 個檔案...")
        create_txt(fn)

    print("\n🎉 所有處理完成！")
else:
    print("❌ 無法取得播放清單，請檢查 URL 是否正確")

## 7. 查看結果

In [ ]:
# 查看下載的檔案
print("📁 下載的音訊檔案:")
audio_files = glob.glob("./download_yt/*")
if audio_files:
    for file in audio_files:
        print(f"  - {os.path.basename(file)}")
else:
    print("  (沒有檔案)")

print("\n📄 生成的文字檔案:")
txt_files = glob.glob("./subtitles/*.txt")
if txt_files:
    for file in txt_files:
        print(f"  - {os.path.basename(file)}")
else:
    print("  (沒有檔案)")

print(f"\n📊 統計:")
print(f"  - 音訊檔案: {len(audio_files)} 個")
print(f"  - 文字檔案: {len(txt_files)} 個")

## 8. 預覽文字內容

In [ ]:
# 預覽生成的文字檔案內容
txt_files = glob.glob("./subtitles/*.txt")

if txt_files:
    for file in txt_files:
        print(f"\n{'='*50}")
        print(f"檔案: {os.path.basename(file)}")
        print(f"{'='*50}")

        with open(file, 'r', encoding='utf-8') as f:
            content = f.read()
            # 只顯示前 500 個字元
            if len(content) > 500:
                print(content[:500] + "\n\n... (內容過長，僅顯示前 500 字元)")
            else:
                print(content)
else:
    print("沒有找到文字檔案，請先執行步驟 5 或 6")

## 9. 下載結果檔案

In [ ]:
# 下載文字檔案到本地
from google.colab import files
import zipfile

txt_files = glob.glob("./subtitles/*.txt")

if txt_files:
    if len(txt_files) == 1:
        # 如果只有一個檔案，直接下載
        print(f"下載檔案: {txt_files[0]}")
        files.download(txt_files[0])
    else:
        # 如果有多個檔案，打包成 zip 檔案
        zip_filename = "subtitles.zip"
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in txt_files:
                zipf.write(file, os.path.basename(file))

        print(f"已打包 {len(txt_files)} 個檔案到 {zip_filename}")
        files.download(zip_filename)
else:
    print("沒有找到文字檔案可供下載")

## 📝 使用說明

### 修改設定
- **URL 修改**: 在步驟 5 中修改 `url_list` 或在步驟 6 中修改 `playlist_url`
- **模型選擇**: 在步驟 3 中可以將 `"medium"` 改為：
  - `"small"` (速度較快，準確度較低)
  - `"large"` (速度較慢，準確度較高)
- **語言設定**: 在 `create_txt` 函數中可以修改 `language="zh"` 或移除讓模型自動偵測

### 注意事項
- 第一次執行會需要下載 Whisper 模型檔案
- 處理時間取決於影片長度和選擇的模型大小
- 建議先用短影片測試
- 如果遇到下載失敗，可能是影片受到地區限制或已被刪除

### 輸出格式
生成的文字檔案包含：
- 時間戳記格式：`[HH:MM:SS --> HH:MM:SS] 文字內容`

### 支援的影片格式
- YouTube 單一影片
- YouTube 播放清單
- 大部分 yt-dlp 支援的網站